# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load .csv and drop empty rows
file = pd.read_csv("cities_weather.csv")
file.dropna()
file.head()

,Unnamed: 0,City,Date,Country,Lat,Lon,Temp,Max Temp,Humidity,Wind Speed,Cloudiness
0,0,Barrow,1586804898,US,71.29,-156.79,23.00,23.00,92,9.17,90
1,1,Albany,1586804651,US,42.60,-73.97,60.39,63.00,82,21.92,90
2,2,Peniche,1586804898,PT,39.36,-9.38,56.28,59.00,62,4.70,40
3,3,Buchanan,1586804870,LR,5.88,-10.05,84.20,84.20,83,9.17,40
4,4,Uvira,1586804898,CD,-3.41,29.15,74.12,74.12,50,0.98,97


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Add gkey to gmaps
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lon' from DF into lists
locations = file[["Lat", "Lon"]]
humidity = file["Humidity"].astype(float)


In [4]:
# Heatmap
HHM_fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Add heat layer over map 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 4)
HHM_fig.add_layer(heat_layer)

#Display figure
HHM_fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Set up what my ideal weather would be
ideal_temp=file.loc[(file['Max Temp']>=70)&(file['Max Temp']<=80)]
ideal_wind=ideal_temp.loc[(ideal_temp['Wind Speed']<=10)]
ideal_weather=ideal_wind.loc[(ideal_wind['Cloudiness']==0)]
ideal_weather

,Unnamed: 0,City,Date,Country,Lat,Lon,Temp,Max Temp,Humidity,Wind Speed,Cloudiness
163,163,San Juan,1586805016,AR,-31.54,-68.54,72.09,72.09,27,6.62,0
221,221,Khorramshahr,1586805161,IR,30.44,48.17,75.54,78.80,43,7.74,0
346,346,Mount Isa,1586805356,AU,-20.73,139.50,71.60,71.60,35,8.05,0
429,429,Chokwé,1586805426,MZ,-24.53,32.98,73.63,73.63,80,6.64,0
522,522,Sūsangerd,1586805565,IR,31.56,48.18,75.20,75.20,31,9.17,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# create hotel file
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# parameters
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}


In [7]:
# Use cities_df location data to identify hotels
for index, row in hotel_df.iterrows():
    
    # Import lats and longs
    lat = row["Lat"]
    lon = row["Lon"]
    
    # Change location, keep parameters during requests to base url
    params["location"] = f"{lat},{lon}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_name = requests.get(base_url, params=params).json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Use an exception to skip errors for places missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing result... skipping.")
hotel_df

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAGytZ95ok4N8SbTqKntlNUztxkz-m-s2ImJdtRYluESCC1OcvGSjVZDXIV_5aPKLG6LkiQXkqgIAVPeLVjInc9WWexvg18KHuJi0py8xp3u8x1g62ODZ5LqT9ESUmGgVCEtJZhk84sBysCZuLX5wXmacAkRWF2S3u6x7tx0LHGSN8eDitWs0cmhEVblq4YNHrzBvG7kGJCWfsQVD5IlcxCpxihXDBHtx3eIy_3hfoXJe2kNoPZ9x602ffsI-Lsyf-qm94FB5VpxVxQ4AMWEUnid_FI14JRCfWDRsfrORKDG1RDdRPTBMdc6VihtsdKzN9BxHEqIbmOpd9UsdNTdnh1kiPnlEjzirpfE04wbrhgdDK6v50lViv7nn1w_SqqwXFxJvjB_NgY_kdKOg-nJpCM4SECwXeoDCa16qPGHalQE0ZIgaFDuBA8fJ5bOAUzMFH23yEIuF3ORf",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -31.5461326,
                    "lng": -68.48806139999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -31.54477622010728,
                        "lng": -68.4867885701073
                    },
                    "southwest": {
                        "lat": -31.54747587989272,
            

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 30.4380661,
                    "lng": 48.2003773
                },
                "viewport": {
                    "northeast": {
                        "lat": 30.43951277989272,
                        "lng": 48.20172607989272
                    },
                    "southwest": {
                        "lat": 30.43681312010727,
                        "lng": 48.19902642010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "ab46a5d0917ed72a1202175c64c5189321c0b7c2",
            "name": "\u0645\u062c\u062a\u0645\u0639 \u0641\u0631\u0647\u0646\u06af\u06cc\u060c \u0631\u0641\u0627\u0647\u06cc \u0648 \u0627\u0642\u0627\u0645\u062a\u06cc \u0633\u0648\u0645 \u062e\u0631\u062f\u0627\u062f",
            "place_id": "ChIJXwEDG7FdxD8R5

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAADDe7JT097ORJD5FltDfqt0etYvSGZbNz7DXFL3haQrA_HqaWlkraWBocpCqOzfSSOP8J9SgrTXGA_3JYKbFLWg30HieYKY4uRSzvUOM9Lf_wmLT78VxNMtnM8Px6fZkAcIkjL1asvvTKfVLR0BpO1U0hz3AD0MR9WCz9KwSgcTkgXmHUgdbt46wN3ZdAdcZ0ha8bGWtcCEGzXwxEBTyfezUXxjWr7cDaToGJMQAxqj6kJyI_LRsW8ZLsmEgmW03jtgGQsoWYoLCougvADicFKr2CA6wCHciBwuV6oW4m7BAFxJBn7QkFquxV9BzdK2dxFVT-fN6ry9NyW-zNNHHIEXk5fBsDaaXgx7C0-XlAGr_BuiK-TlTlGB29JIXrOdaQbcEilHtQPRMtKTMEBFzwVASEM9lx7fAJLDa3Rw7fNL5FyIaFGGqNFuLKVs0tm1PaIE6THL-YKdK",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -20.6994709,
                    "lng": 139.4867777
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.69806627010728,
                        "lng": 139.4876622298927
                    },
                    "southwest": {
                        "lat": -20.70076592989272,
                   

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -24.5448163,
                    "lng": 33.0063827
                },
                "viewport": {
                    "northeast": {
                        "lat": -24.54352007010727,
                        "lng": 33.00752592989272
                    },
                    "southwest": {
                        "lat": -24.54621972989272,
                        "lng": 33.00482627010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "d97711aaaf9bab644193243df117cb24dc2b6828",
            "name": "Hotel Sorriso de Caridade",
            "place_id": "ChIJWXtugr_93R4RLPfAMu4enwU",
            "plus_code": {
                "compound_code": "F244+3H Chokwe, Mozambique",
                "global_code": "5G7MF244+3H"
            },
       

,Unnamed: 0,City,Date,Country,Lat,Lon,Temp,Max Temp,Humidity,Wind Speed,Cloudiness,Hotel Name
0,163,San Juan,1586805016,AR,-31.54,-68.54,72.09,72.09,27,6.62,0,Hotel Viñas del Sol
1,221,Khorramshahr,1586805161,IR,30.44,48.17,75.54,78.80,43,7.74,0,مجتمع فرهنگی، رفاهی و اقامتی سوم خرداد
2,346,Mount Isa,1586805356,AU,-20.73,139.50,71.60,71.60,35,8.05,0,Abacus Motel
3,429,Chokwé,1586805426,MZ,-24.53,32.98,73.63,73.63,80,6.64,0,Hotel Sorriso de Caridade
4,522,Sūsangerd,1586805565,IR,31.56,48.18,75.20,75.20,31,9.17,0,"Hall, Hotel And Restaurant Ghasr"


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

HHM_fig.add_layer(markers)

# Display Map
HHM_fig

Figure(layout=FigureLayout(height='420px'))